In [70]:
from typing import List, Tuple, Dict
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

# Ensure necessary NLTK tokenizer models are available
nltk.download("punkt")
nltk.download('punkt_tab')

class Tokenizer:
    def __init__(self, tokenize_type: str = "basic", lowercase: bool = False):
        self.lowercase = lowercase
        self.type = tokenize_type
        self.vocab = []  # Empty vocabulary list

    def basicTokenize(self, string: str) -> List[str]:
        # Tokenizes input string by splitting on whitespace
        ### BEGIN SOLUTION
        return string.split()
        ### END SOLUTION

    def nltkTokenize(self, string: str) -> List[str]:
        # Tokenizes input string using NLTK's word tokenizer
        ### BEGIN SOLUTION
        return word_tokenize(string)
        ### END SOLUTION

    def tokenize(self, string: str) -> List[str]:
        # Tokenizes string and updates vocabulary with unique words
        if self.lowercase:
            string = string.lower()
        tokens = self.basicTokenize(string) if self.type == "basic" else self.nltkTokenize(string)
        self.vocab += [w for w in set(tokens) if w not in self.vocab]
        return tokens

    def countTopWords(self, words: List[str], k: int) -> List[Tuple[str, int]]:
        # Returns the top k most common words
        ### BEGIN SOLUTION
        word_counts = Counter(words)
        return word_counts.most_common(k)
        ### END SOLUTION

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [71]:
class BiGramLanguageModel:
    def __init__(self, vocab: List[str], smoothing: str = None, smoothing_param: float = None): pass
    def computeBigramProb(self): pass
    def computeBigramProbAddAlpha(self, alpha: float = 0.001): pass
    def train(self, corpus: List[str]): pass
    def test(self, corpus: List[str]) -> float: pass

In [72]:
def bigram_init(self, vocab, smoothing=None, smoothing_param=None):
    self.vocab = vocab
    self.token_to_idx = {word: i for i, word in enumerate(vocab)}
    self.smoothing = smoothing
    self.smoothing_param = smoothing_param
    self.bi_counts = None
    self.bi_prob = None
    assert smoothing is None or smoothing_param is not None, "Smoothing parameters must be set correctly."

In [73]:
def bigram_compute_prob(self):
    ### BEGIN SOLUTION
    self.bi_prob = np.zeros_like(self.bi_counts)
    for i in range(len(self.vocab)):
        row_sum = self.bi_counts[i].sum()
        if row_sum > 0:
            self.bi_prob[i] = self.bi_counts[i] / row_sum
        else:
            self.bi_prob[i] = 0.0




    ### END SOLUTION

In [74]:
def bigram_compute_prob_add_alpha(self, alpha=1e-3):
    self.bi_prob = np.zeros_like(self.bi_counts)
    for i in range(len(self.vocab)):
        row_sum = self.bi_counts[i].sum()
        denominator = row_sum + alpha * len(self.vocab)
        self.bi_prob[i] = (self.bi_counts[i] + alpha) / denominator

In [75]:
def bigram_train(self, corpus):
    self.bi_counts = np.zeros((len(self.vocab), len(self.vocab)), dtype=float)
    corpus_indices = [self.token_to_idx[w] for w in corpus]
    for i in range(len(corpus_indices) - 1):
        self.bi_counts[corpus_indices[i]][corpus_indices[i + 1]] += 1
    if self.smoothing == "addAlpha":
        self.computeBigramProbAddAlpha(self.smoothing_param)
    else:
        self.computeBigramProb()

In [76]:
def bigram_test(self, corpus):
    logprob = 0.0
    eps = 1e-10
    corpus_indices = [self.token_to_idx[w] for w in corpus]
    for i in range(len(corpus_indices) - 1):
        prob = self.bi_prob[corpus_indices[i], corpus_indices[i + 1]]
        logprob += np.log(prob + eps)
    logprob /= len(corpus_indices) - 1
    return np.exp(-logprob)

In [77]:
BiGramLanguageModel.__init__ = bigram_init
BiGramLanguageModel.computeBigramProb = bigram_compute_prob
BiGramLanguageModel.computeBigramProbAddAlpha = bigram_compute_prob_add_alpha
BiGramLanguageModel.train = bigram_train
BiGramLanguageModel.test = bigram_test

In [78]:
def readCorpus(filename: str, tokenizer: Tokenizer) -> List[str]:
    # Reads and tokenizes the corpus from a file
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
    return tokenizer.tokenize(text)

In [79]:
def runLanguageModel(train_corpus: List[str], val_corpus: List[str], tokenizer: Tokenizer, smoothing_type: str = None, smoothing_param: float = 0.0) -> Dict[str, float]:
    # Trains and tests the language model, returning key metrics
    lm = BiGramLanguageModel(tokenizer.vocab, smoothing=smoothing_type, smoothing_param=smoothing_param)
    lm.train(train_corpus)
    return {"train_ppl": lm.test(train_corpus), "val_ppl": lm.test(val_corpus)}

In [80]:
# Initialize tokenizers with basic and NLTK options, both set to lowercase.
basic_tokenizer = Tokenizer(tokenize_type='basic', lowercase=True)
nltk_tokenizer = Tokenizer(tokenize_type='nltk', lowercase=True)

In [81]:
# Read and tokenize the training and validation corpora using the basic tokenizer.
train_corpus = readCorpus('./data/train.txt', basic_tokenizer)
val_corpus = readCorpus('./data/val.txt', basic_tokenizer)

# Example of using the NLTK tokenizer for comparison (unused in final results).
train_corpus_nltk = readCorpus('./data/train.txt', nltk_tokenizer)
val_corpus_nltk = readCorpus('./data/val.txt', nltk_tokenizer)

In [82]:
# Get top 10 frequent words and counts from train_corpus with basic_tokenizer.
### BEGIN PUBLIC TESTS
basic_tokenizer.countTopWords(train_corpus, k=10)
### END PUBLIC TESTS

[('unk', 61019),
 ('the', 45302),
 ('of', 25379),
 ('and', 18067),
 ('to', 16515),
 ('a', 14371),
 ('in', 14231),
 ('is', 7466),
 ('that', 6484),
 ('for', 6434)]

In [83]:
# Get top 10 frequent words and counts from train_corpus_nltk with nltk_tokenizer.
### BEGIN PUBLIC TESTS
nltk_tokenizer.countTopWords(train_corpus_nltk, k=10)
### END PUBLIC TESTS

[('unk', 61019),
 ('the', 45885),
 ('of', 25427),
 (',', 23570),
 ('and', 18346),
 ('.', 17532),
 ('to', 16606),
 ('a', 14721),
 ('in', 14358),
 ('is', 7702)]

In [84]:
# Run the language model with the basic tokenizer and without smoothing.
### BEGIN PUBLIC TESTS
runLanguageModel(train_corpus, val_corpus,
                 tokenizer=basic_tokenizer)
### END PUBLIC TESTS

{'train_ppl': np.float64(69.87840111738812),
 'val_ppl': np.float64(58435.97213541714)}

In [85]:
# Run the language model with the nltk tokenizer and without smoothing.
### BEGIN PUBLIC TESTS
runLanguageModel(train_corpus_nltk, val_corpus_nltk,
                 tokenizer=nltk_tokenizer)
### END PUBLIC TESTS

{'train_ppl': np.float64(69.68966075535424),
 'val_ppl': np.float64(9285.574606771734)}

In [86]:
# Run the language model with the basic tokenizer and with smoothing.
runLanguageModel(train_corpus, val_corpus,
                 tokenizer=basic_tokenizer, smoothing_type='addAlpha', smoothing_param=10e-5)

{'train_ppl': np.float64(74.15013253687664),
 'val_ppl': np.float64(2216.4589207208182)}

In [87]:
# Run the language model with the nltk tokenizer and with smoothing.
runLanguageModel(train_corpus_nltk, val_corpus_nltk,
                 tokenizer=nltk_tokenizer, smoothing_type='addAlpha', smoothing_param=10e-5)

{'train_ppl': np.float64(71.60376054214575),
 'val_ppl': np.float64(914.026630299532)}